In [1]:
import numpy as np
import pandas as pd
from simulate_matches import simulate_matches

## Partida 1
Nessa partida, $p=0.75$ e, consequentemente, $q=0.25$

In [2]:
simulate_matches('matches_1.csv', 0.7, 120)
df_1 = pd.read_csv('matches_1.csv')

120 0


In [7]:
def matches_won(df : pd.DataFrame):
    a_matches = 0
    b_matches = 0

    for match in df['match'].unique():
        df_match = df[df['match'] == match]

        a_sets, b_sets = sets_won(df_match)

        if (a_sets == 2 and b_sets == 0) or (a_sets == 2 and b_sets == 1):
            a_matches += 1
        else:
            b_matches += 1
    
    return a_matches, b_matches

a, b = matches_won(df_1)

print(f'A won {100*a/(a+b):.2f}% of the matches')
print(f'B won {100*b/(a+b):.2f}% of the matches')

A won 100.00% of the matches
B won 0.00% of the matches


In [6]:
def sets_won(df : pd.DataFrame):
    a_sets = 0
    b_sets = 0

    for match in df['match'].unique():
        df_match = df[df['match'] == match]
        for set in df_match['set'].unique():
            df_set = df_match[df_match['set'] == set]
            a_games, b_games = games_won(df_set)

            if a_games == 7 or (a_games == 6 and b_games < 6):
                a_sets += 1
            else:
                b_sets += 1
        
    return a_sets, b_sets

a, b = sets_won(df_1)

print(f'A won {100*a/(a+b):.2f}% of the sets')
print(f'B won {100*b/(a+b):.2f}% of the sets')

A won 100.00% of the sets
B won 0.00% of the sets


In [5]:
def games_won(df : pd.DataFrame):
    a_games = len(df[df['state'] == 'win_a'])
    b_games = len(df[df['state'] == 'win_b'])
    return a_games, b_games

a, b = games_won(df_1)

print(f'A won {100*a/(a+b):.2f}% of the games')
print(f'B won {100*b/(a+b):.2f}% of the games')

A won 89.67% of the games
B won 10.33% of the games


In [4]:
def points_scored(df : pd.DataFrame):
    a_points = len(df[df['point'] == 'a'])
    b_points = len(df[df['point'] == 'b'])
    return a_points, b_points

a, b = points_scored(df_1)

print(f'A scored {100*a/(a+b):.2f}% of the points')
print(f'B scored {100*b/(a+b):.2f}% of the points')

A scored 70.06% of the points
B scored 29.94% of the points


## Partida 2
Agora, $p=0.55$ e $q=0.45$

In [16]:
simulate_matches('matches_2.csv', 0.5, 120)
df_2 = pd.read_csv('matches_2.csv')

59 61


In [74]:
a, b = games_won(df_2)

print(f'A won {100*a/(a+b):.2f}% of the games')
print(f'B won {100*b/(a+b):.2f}% of the games')

A won 50.31% of the games
B won 49.69% of the games


In [99]:
a, b = sets_won(df_2)

print(f'A won {100*a/(a+b):.2f}% of the sets')
print(f'B won {100*b/(a+b):.2f}% of the sets')

A won 46.47% of the sets
B won 53.53% of the sets


In [212]:
a, b = points_scored(df_2)

print(f'A scored {100*a/(a+b):.2f}% of the points')
print(f'B scored {100*b/(a+b):.2f}% of the points')

A scored 55.13% of the points
B scored 44.87% of the points


In [11]:
a, b = matches_won(df_2)

print(f'A won {100*a/(a+b):.2f}% of the matches')
print(f'B won {100*b/(a+b):.2f}% of the matches')

A won 85.00% of the matches
B won 15.00% of the matches
